# Lab 2

## Actividades

1. Crear tu propio entorno y entrenar agentes RL en el mismo. Analizar la convergencia con distintos algoritmos* (ej: PPO, DQN), resultados con distintas funciones de recompensa e híper-parámetros. 

    Algunas ideas:

    * Transformar GoLeftEnv en una grilla 2D, añadir paredes / trampas / agua.
    * Crear un entorno que juegue a algún juego como el ta-te-ti.
    * Crea un entorno totalmente nuevo que sea de tu interés!

2. Entrena agentes en entornos más complejos con stable-baselines/rl-baselines-zoo. Tener en cuenta:

    * Google Colab tiene una limitante en cuanto a cantidad de recursos de CPU/GPU (incluido un "rendimiento decreciente silencioso"), lo cuál reduce la capacidad de entrenar distintos entornos.
    * Si el entorno no está implementado en stable-baselines, debe hacerse un wrapper a mano, lo que puede ser sencillo o puede llevar algo más de trabajo, teniendo que tocar código subyacente de la librería. 

\* pueden ser usando stable-baselines/rl-baselines-zoo o bien utilizando algún otro algoritmo (incluso tabular)

## Ejercicio 1: entorno personalizado - Tiro parabólico

A continuación se presenta un entorno en donde el agente debe aprender cuál es el ángulo óptimo para tener un alcance igual al de un objetivo predefinido. La fórmula física para el tiro parabólico es la siguiente:

$X=\frac{v_{i}^{2}sen(2\alpha)}{g}$

En donde $v_{i}^{2}$ es la velocidad inicial, $\alpha$ el ángulo de tiro y $g$ es la gravedad.

El agente inicia con un ángulo aleatorio (entre varios múltiplos de pi/180, para mayor simplicidad) y puede ejecutar 4 acciones: dos aumentan y dos disminuyen el ángulo de tiro, de manera brusca o fina. El agente percibe recompensa al tener el alcance igual al objetivo, y tiene recompensa negativa por cada paso que no llega a dicho ángulo.

Aplicamos PPO y A2C y observamos el desempeño.

### Importación de librerías

In [1]:
import os
from subprocess import Popen, PIPE

import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import spaces
#from gym.envs.registration import register

import random

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.env_util import make_vec_env

os.makedirs('logs', exist_ok=True)

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

cwd = os.getcwd()

%matplotlib inline
%load_ext tensorboard

### Definición del ambiente

In [2]:
class TiroParabolico(gym.Env):
  """
  Ambiente personalizado que sigue la interfaz de gym.
  Es un entorno en el cual el agente debe aprender el a obtener el 
  ángulo adecuado para llegar a un objetivo mediante un tiro parabólico,
  a una velocidad constante de 60 m/s.
  """
  # Dado que estamos en colab, no podemos implementar la salida por interfaz 
  # gráfica ('human' render mode) 
  metadata = {'render.modes': ['console']}

  def __init__(self):
    super(TiroParabolico, self).__init__()
    
    # Velocidad inicial
    self.init_speed = 60
    # Ángulos posibles para el cual se calcula la posición del objetivo
    self.possible_angles = np.arange(np.pi/180, np.pi/2, np.pi/45)
    # Posiciones del objetivo posibles, de acuerdo a los ángulos posibles
    self.possible_x = ((self.init_speed ** 2) * np.sin(2*self.possible_angles)) / 9.8
    
    # 4 acciones: aumentar o disminuir el ángulo, de manera 
    # brusca o fina
    n_actions = 4
    self.action_space = spaces.Discrete(n_actions)
    
    # Espacio de observacion continuo: entre 0 y pi/2 (90 grados sexagesimales)
    self.observation_space = spaces.Box(0, np.pi/2, shape=(1,), dtype=np.float32)

  def reset(self):
    """
    Importante: la observación devuelta debe ser un array de numpy
    :return: (np.array) 
    """
    self.angle = random.choice(self.possible_angles)
    self.target_x = random.choice(self.possible_x)
    
    return np.array(self.angle).astype(np.float32)

  def step(self, action):
    
    # Si el ángulo se va de rango (mayor a pi/2 o menor a 0),
    # que defina un nuevo ángulo aleatoriamente
    # Sino, que aumente o disminuya el ángulo
    # de manera brusca (pi/9) o fina (pi/180)
    if (self.angle >= np.pi/2) | (self.angle <= 0):
        #done = True
        self.angle = random.choice(self.possible_angles)
    else:
        if action == 0:
          self.angle -= np.pi/9
        elif action == 1:
          self.angle -= np.pi/180
        elif action == 2:
          self.angle += np.pi/9
        elif action == 3:
          self.angle += np.pi/180
    
    # Alcance obtenido con el ángulo elegido
    self.alcance = ((self.init_speed ** 2) * np.sin(2*self.angle)) / 9.8
    
    # Recompensa positiva al llegar al objetivo, o negativa
    # en cada accion si no llega. Penaliza que recorra
    # muchos ángulos antes de obtener el óptimo
    if self.alcance == self.target_x:
        reward = 100
        done = True
    else:
        reward = -1
        done = False
        
    info = {}

    return np.array([self.angle]).astype(np.float32), reward, done, info

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # en nuestra interfaz de consola, presentamos el alcance obtenido
    print("Posicion objetivo =", self.target_x, "Alcance aprendido =", self.alcance)

  def close(self):
    pass

### Aprendizaje con PPO

In [3]:
# PPO

env = TiroParabolico()
env = make_vec_env(lambda: env, n_envs=1)

model = PPO('MlpPolicy', env, verbose=0, tensorboard_log='tensorboard_c/', seed = 42).learn(500000)

/home/pablo/miniconda3/envs/diplodatos/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# A2C

env = TiroParabolico()
env = make_vec_env(lambda: env, n_envs=1)

model = A2C('MlpPolicy', env, verbose=0, tensorboard_log='tensorboard_c/', seed = 42).learn(500000)

In [5]:
if not IN_COLAB:
    obs = env.reset()
    for i in range(100):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        env.render(mode='console')
        if dones[0]:
            obs = env.reset()

Posicion objetivo = 281.40408114574706 Alcance aprendido = 245.8030798869274
Posicion objetivo = 281.40408114574706 Alcance aprendido = 12.820223278469644
Posicion objetivo = 281.40408114574706 Alcance aprendido = -226.16135828289495
Posicion objetivo = 281.40408114574706 Alcance aprendido = 353.11654136509674
Posicion objetivo = 281.40408114574706 Alcance aprendido = 335.5881272972819
Posicion objetivo = 281.40408114574706 Alcance aprendido = 161.03429882047737
Posicion objetivo = 281.40408114574706 Alcance aprendido = -88.86926777130658
Posicion objetivo = 281.40408114574706 Alcance aprendido = 12.820223278469744
Posicion objetivo = 281.40408114574706 Alcance aprendido = 245.8030798869275
Posicion objetivo = 281.40408114574706 Alcance aprendido = 363.7719436193523
Posicion objetivo = 281.40408114574706 Alcance aprendido = 311.5278720574626
Posicion objetivo = 281.40408114574706 Alcance aprendido = 113.51644691324601
Posicion objetivo = 281.40408114574706 Alcance aprendido = -137.6105

A continuación se muestra un ejemplo de cómo el agente llega a la posición objetivo (PPO), ajustando el ángulo:

Posicion objetivo = 311.5278720574626 Alcance aprendido = 63.78912648989275

Posicion objetivo = 311.5278720574626 Alcance aprendido = -183.67346938775515

Posicion objetivo = 311.5278720574626 Alcance aprendido = 245.8030798869274

Posicion objetivo = 311.5278720574626 Alcance aprendido = 12.820223278469644

Posicion objetivo = 311.5278720574626 Alcance aprendido = -226.16135828289495

Posicion objetivo = 311.5278720574626 Alcance aprendido = 335.5881272972819

Posicion objetivo = 311.5278720574626 Alcance aprendido = 161.03429882047737

Posicion objetivo = 311.5278720574626 Alcance aprendido = -88.86926777130658

Posicion objetivo = 311.5278720574626 Alcance aprendido = 12.820223278469744

Posicion objetivo = 311.5278720574626 Alcance aprendido = 245.8030798869275

Posicion objetivo = **311.5278720574626** Alcance aprendido = 363.7719436193523

Posicion objetivo = 63.789126489892766 Alcance aprendido = **311.5278720574626**

In [6]:
%tensorboard --logdir=tensorboard_c/

Reusing TensorBoard on port 6006 (pid 8131), started 0:00:54 ago. (Use '!kill 8131' to kill it.)

## Capturas de pantalla de tensorboard

Recompensa percibida por episodio

PPO

![env_reward](env_reward.png)

A2C

![env_reward](env_reward2.png)

Número de pasos por episodio

PPO

![env_pasos](env_ep_len.png)

A2C

![env_pasos](env_ep_len2.png)

### Comentarios finales

Un análisis preliminar indica que PPO se desempeña mejor que A2C. Sin embargo, se podría profundizar en la búsqueda de un mejor método de recompensa y en un ajuste de hiperparámetros para obtener una curva de recompensa más consistente.

## Ejercicio 2: entorno complejo con stable-baselines/rl-baselines-zoo

Se ejecutó el ambiente "Pendulum-v0" mediante la terminal de Linux (una vez clonado localmente el repositorio stable-baselines/rl-baselines-zoo). En este ambiente, el agente debe aprender a posicionar el péndulo de manera vertical. A continuación, se detallan las ejecuciones en la terminal y se muestran las salidas gráficas obtenidas. 

### Ejecución del aprendizaje con PPO

python train.py --algo ppo --env Pendulum-v0 --tensorboard-log /tmp/stable-baselines/

### Salida final del aprendizaje

![outputfinal](ultimacorrida.png)

### Salida gráfica: recompensa obtenida por episodio

python scripts/plot_train.py -a ppo -y reward --env Pendulum-v0_1 -f logs/ -x steps

![reward](reward.png)

### Captura de pantalla del agente aprendido

python enjoy.py --algo ppo --env Pendulum-v0 -f logs/ --exp-id 0

A la derecha, se observa que el péndulo ha llegado al objetivo de estar vertical sin caerse.

![enjoy](enjoy2.png)
![enjoy2](enjoy1.png)